In [1]:
import pandas as pd

In [10]:
ped_data = pd.read_csv('../../data/pediatric_patients_report_drug_reaction.csv.gz',compression='gzip', index_col=0)

In [11]:
ped_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 881396 entries, 1 to 1754669
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   patient_custom_master_age       881396 non-null  float64
 1   patient_custom_master_age_unit  881396 non-null  object 
 2   patient_onsetage                881396 non-null  float64
 3   patient_onsetageunit            881396 non-null  object 
 4   patient_sex                     881396 non-null  object 
 5   safetyreportid                  881396 non-null  object 
 6   nichd                           881396 non-null  object 
 7   ich_ema                         639944 non-null  object 
 8   fda                             529859 non-null  object 
 9   lastupdate_date                 881396 non-null  object 
 10  mostrecent_receive_date         881396 non-null  object 
 11  receive_date                    881396 non-null  object 
 12  congenital_anomali  

In [12]:
# unique_drugs_per_report = ped_data.groupby('safetyreportid')['medicinal_product'].nunique().reset_index()
# unique_drugs_per_report.columns = ['safetyreportid', 'unique_drug_count']

In [13]:
# unique_drugs_per_report.sort_values(by='unique_drug_count', ascending=True)

In [14]:
concept = (pd.read_csv('../../data/openFDA_drug_event/vocabulary_download_v5/CONCEPT.csv',sep='\t',
                      dtype={
                          'concept_id' : 'int'
                      }))
concept.head()

/tmp/ipykernel_22822/1850678616.py:1: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concept = (pd.read_csv('../../data/openFDA_drug_event/vocabulary_download_v5/CONCEPT.csv',sep='\t',


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN


In [15]:
concept_relationship = (pd.
                        read_csv('../../data/openFDA_drug_event/vocabulary_download_v5/'+
                                 'CONCEPT_RELATIONSHIP.csv',sep='\t',
                                dtype={
                                    'concept_id_1' : 'int',
                                    'concept_id_2' : 'int'
                                }))
concept_relationship.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,21014172,21034689,RxNorm dose form of,20150817,20991231,NaN
1,21014172,21044546,RxNorm dose form of,20150817,20991231,NaN
2,21014172,21054337,RxNorm dose form of,20150817,20991231,NaN
3,21014172,21064199,RxNorm dose form of,20150817,20991231,NaN
4,21014172,21095576,RxNorm dose form of,20150817,20991231,NaN


In [16]:
print("--- ขั้นตอนที่ 1: กำลังทำ Standard MedDRA ---")

# 1.1 เตรียมข้อมูล MedDRA Preferred Term (PT) จากตาราง CONCEPT
print("กำลังเตรียม MedDRA concepts...")
meddra_concepts = concept.query("vocabulary_id == 'MedDRA' and concept_class_id == 'PT'").copy()

# Normalizing ชื่ออาการให้เป็น Title Case เพื่อให้ join ได้ง่ายขึ้น
meddra_concepts['concept_name_normalized'] = meddra_concepts['concept_name'].str.title()

# เลือกและเปลี่ยนชื่อคอลัมน์เพื่อป้องกันการซ้ำซ้อน
meddra_for_join = meddra_concepts[[
    'concept_id', 'concept_name_normalized', 'concept_code'
]].rename(columns={
    'concept_id': 'MedDRA_concept_id',
    'concept_name_normalized': 'MedDRA_concept_name',
    'concept_code': 'MedDRA_concept_code'
})

# 1.2 เตรียมข้อมูลในตารางหลัก
# สร้างคอลัมน์ชั่วคราวสำหรับ join
ped_data['reaction_meddrapt_normalized'] = ped_data['reaction_meddrapt'].str.title()

# 1.3 ทำการ Merge (Join)
print("กำลังเชื่อมโยงข้อมูลอาการกับ MedDRA ID...")
ped_data_with_meddra = pd.merge(
    ped_data,
    meddra_for_join,
    left_on='reaction_meddrapt_normalized',
    right_on='MedDRA_concept_name',
    how='left'  # <<-- ใช้ 'left' join เพื่อเก็บข้อมูลเด็กทั้งหมดไว้
)

# 1.4 ทำความสะอาดข้อมูลหลัง Merge
ped_data_with_meddra = ped_data_with_meddra.drop(columns=['reaction_meddrapt_normalized', 'MedDRA_concept_name'])

print("\nการเชื่อมโยง MedDRA เสร็จสิ้น!")
# ตรวจสอบผลลัพธ์
meddra_mapped_count = ped_data_with_meddra['MedDRA_concept_id'].notna().sum()
total_records = len(ped_data_with_meddra)
print(f"จำนวน records ทั้งหมด: {total_records}")
print(f"จำนวน records ที่เชื่อม MedDRA ID ได้: {meddra_mapped_count} ({meddra_mapped_count/total_records:.2%})")

print("\nตัวอย่างข้อมูลหลังเพิ่ม MedDRA:")
print(ped_data_with_meddra[['safetyreportid', 'reaction_meddrapt', 'MedDRA_concept_id']].head())

--- ขั้นตอนที่ 1: กำลังทำ Standard MedDRA ---
กำลังเตรียม MedDRA concepts...


กำลังเชื่อมโยงข้อมูลอาการกับ MedDRA ID...

การเชื่อมโยง MedDRA เสร็จสิ้น!
จำนวน records ทั้งหมด: 881396
จำนวน records ที่เชื่อม MedDRA ID ได้: 811930 (92.12%)

ตัวอย่างข้อมูลหลังเพิ่ม MedDRA:
  safetyreportid          reaction_meddrapt  MedDRA_concept_id
0       25129349  Bundle branch block right         35204912.0
1       25129349  Bundle branch block right         35204912.0
2       25129367          Treatment failure         35809361.0
3       25129367          Treatment failure         35809361.0
4       25129367          Treatment failure         35809361.0


In [24]:
er_dir = '../../data/openFDA_drug_event/er_tables_memory_efficient/'

print("\n--- ขั้นตอนที่ 2 (ฉบับแก้ไข KeyError ครั้งสุดท้าย): กำลังทำ Standard ATC ผ่าน Ingredient ---")

# --- ส่วน A: สร้างตารางเชื่อมโยงที่จำเป็นขึ้นมาใหม่ (ส่วนนี้ถูกต้องแล้ว) ---
# (นำโค้ดส่วน A จากคำตอบก่อนหน้ามาวางที่นี่ได้เลย หรือรันส่วนด้านล่างนี้)

print("A.1: กำลังสร้างตาราง Product-to-Ingredient map...")
has_ingredient_rels = concept_relationship.query("relationship_id in ['Has precise ing', 'Has ingredient']").copy()
rx_products = concept.query("vocabulary_id == 'RxNorm' and concept_class_id not in ['Ingredient', 'Brand Name']").copy()
rx_ingredients = concept.query("vocabulary_id == 'RxNorm' and concept_class_id == 'Ingredient'").copy()

product_to_ingredient_map = pd.merge(
    has_ingredient_rels,
    rx_products[['concept_id']].add_prefix('product_'),
    left_on='concept_id_1',
    right_on='product_concept_id'
)
product_to_ingredient_map = pd.merge(
    product_to_ingredient_map,
    rx_ingredients[['concept_id', 'concept_name']].add_prefix('ingredient_'),
    left_on='concept_id_2',
    right_on='ingredient_concept_id'
)
product_to_ingredient_map = product_to_ingredient_map[['product_concept_id', 'ingredient_concept_id', 'ingredient_concept_name']].drop_duplicates()
product_to_ingredient_map = product_to_ingredient_map.rename(columns={'ingredient_concept_name': 'ingredient_name'})
print(f"สร้าง Product-to-Ingredient map สำเร็จ")

print("A.2: กำลังสร้างตาราง Ingredient-to-ATC map...")
atc_concepts = concept.query("vocabulary_id == 'ATC' and concept_class_id == 'ATC 5th'").copy()
atc_for_join = atc_concepts[['concept_id', 'concept_code', 'concept_name']].rename(columns={
    'concept_id': 'ATC_concept_id', 'concept_code': 'ATC_concept_code', 'concept_name': 'ATC_concept_name'
})
rx_ingredients_for_join = rx_ingredients[['concept_id', 'concept_name']].rename(columns={
    'concept_id': 'ingredient_concept_id', 'concept_name': 'ingredient_name'
})
maps_to_relations = concept_relationship.query("relationship_id == 'Maps to' and invalid_reason != 'U'").copy()

ingredient_to_atc_map = pd.merge(maps_to_relations, atc_for_join, left_on='concept_id_2', right_on='ATC_concept_id')
ingredient_to_atc_map = pd.merge(ingredient_to_atc_map, rx_ingredients_for_join, left_on='concept_id_1', right_on='ingredient_concept_id')
ingredient_to_atc_map = ingredient_to_atc_map[['ingredient_concept_id', 'ingredient_name', 'ATC_concept_id', 'ATC_concept_code', 'ATC_concept_name']].drop_duplicates()
print(f"สร้าง Ingredient-to-ATC map สำเร็จ")


# --- ส่วน B: เชื่อมโยงข้อมูลทั้งหมดเข้าด้วยกัน (ส่วนที่แก้ไขอย่างละเอียด) ---
print("\nB.1: กำลังเชื่อมโยงข้อมูลยา -> RxNorm -> Ingredient -> ATC...")

# B.1.1: โหลด standard_drugs และกรองเฉพาะข้อมูลเด็ก
standard_drugs = pd.read_csv(
    er_dir + 'standard_drugs.csv.gz',
    compression='gzip',
    dtype={'safetyreportid': 'str', 'RxNorm_concept_id': 'int'}
)
ped_reports = ped_data_with_meddra['safetyreportid'].unique()
pediatric_standard_drugs = standard_drugs[standard_drugs['safetyreportid'].isin(ped_reports)].copy()

# B.1.2: Merge ตารางหลักกับ RxNorm Product ID (*** จุดที่แก้ไข ***)
# สร้าง join key จากชื่อยาในแต่ละตาราง
ped_data_with_meddra['join_drug_name'] = ped_data_with_meddra['medicinal_product'].str.lower()
pediatric_standard_drugs['join_drug_name'] = pediatric_standard_drugs['RxNorm_concept_name'].str.lower()

final_dataset = pd.merge(
    ped_data_with_meddra,
    pediatric_standard_drugs.rename(columns={'RxNorm_concept_id': 'RxNorm_product_id'}),
    on=['safetyreportid', 'join_drug_name'], # <-- Join ด้วย safetyreportid และชื่อยาที่แปลงแล้ว
    how='left'
)

# B.1.3: Merge กับ Product-to-Ingredient Map
final_dataset = pd.merge(
    final_dataset,
    product_to_ingredient_map,
    left_on='RxNorm_product_id',
    right_on='product_concept_id',
    how='left'
)

# B.1.4: Merge กับ Ingredient-to-ATC Map
final_dataset = pd.merge(
    final_dataset,
    ingredient_to_atc_map,
    on='ingredient_concept_id',
    how='left',
    suffixes=('', '_from_atc_map')
)

# B.1.5: ทำความสะอาดข้อมูล
# ลบคอลัมน์ที่ไม่จำเป็นและจัดการคอลัมน์ที่ซ้ำซ้อน
cols_to_drop = [
    'join_drug_name', 'RxNorm_product_id', 'product_concept_id',
    'ingredient_name_from_atc_map', 'RxNorm_concept_code',
    'RxNorm_concept_name', 'RxNorm_concept_class_id'
]
final_dataset = final_dataset.drop(columns=[col for col in cols_to_drop if col in final_dataset.columns], errors='ignore')
final_dataset.drop_duplicates(inplace=True)


print("\nการเชื่อมโยง ATC เสร็จสิ้น!")
atc_mapped_count = final_dataset['ATC_concept_id'].notna().sum()
total_records_final = len(final_dataset)
print(f"จำนวน records ทั้งหมดในตารางสุดท้าย: {total_records_final}")
print(f"จำนวน records ที่เชื่อม ATC ID ได้: {atc_mapped_count} (คิดเป็นประมาณ {atc_mapped_count/total_records_final:.2%})")

print("\nตัวอย่างข้อมูลสุดท้าย:")
print(final_dataset[[
    'safetyreportid', 'medicinal_product', 'reaction_meddrapt',
    'MedDRA_concept_id', 'ingredient_name', 'ATC_concept_name'
]].dropna(subset=['ATC_concept_name']).head(10))


--- ขั้นตอนที่ 2 (ฉบับแก้ไข KeyError ครั้งสุดท้าย): กำลังทำ Standard ATC ผ่าน Ingredient ---
A.1: กำลังสร้างตาราง Product-to-Ingredient map...
สร้าง Product-to-Ingredient map สำเร็จ
A.2: กำลังสร้างตาราง Ingredient-to-ATC map...
สร้าง Ingredient-to-ATC map สำเร็จ

B.1: กำลังเชื่อมโยงข้อมูลยา -> RxNorm -> Ingredient -> ATC...

การเชื่อมโยง ATC เสร็จสิ้น!
จำนวน records ทั้งหมดในตารางสุดท้าย: 881324
จำนวน records ที่เชื่อม ATC ID ได้: 0 (คิดเป็นประมาณ 0.00%)

ตัวอย่างข้อมูลสุดท้าย:
Empty DataFrame
Columns: [safetyreportid, medicinal_product, reaction_meddrapt, MedDRA_concept_id, ingredient_name, ATC_concept_name]
Index: []
